# -----import package-----

In [56]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
from shapely.ops import linemerge
from shapely.wkt import loads
from shapely.geometry import LineString
from shapely.geometry import Point, Polygon
from scipy.spatial import Delaunay

# -----point file-----

In [2]:
df_point = gpd.read_file('hydrant_location/hydrant_location.shp')

In [3]:
df_point['id'] = df_point.reset_index().index

In [4]:
df_point

,id,geometry
0,0,POINT (-74.00698 40.74748)
1,1,POINT (-74.00722 40.74722)
2,2,POINT (-74.00721 40.74697)
3,3,POINT (-74.00722 40.74722)
4,4,POINT (-74.00728 40.74667)
...,...,...
2295,2295,POINT (-73.97976 40.71559)
2296,2296,POINT (-73.97987 40.71361)
2297,2297,POINT (-73.98387 40.72077)
2298,2298,POINT (-73.97780 40.71833)


# -----road file-----

In [5]:
df_road = gpd.read_file('road_location/road_location.shp')

In [6]:
df_road.drop('road_id', axis=1, inplace=True)

In [7]:
df_road

,id,geometry
0,1,"LINESTRING (-74.01301 40.70215, -74.01167 40.7..."
1,1,"LINESTRING (-74.01171 40.70247, -74.00999 40.7..."
2,1,"LINESTRING (-74.00577 40.70566, -74.00396 40.7..."
3,1,"LINESTRING (-74.00385 40.70651, -74.00142 40.7..."
4,2,"LINESTRING (-74.01270 40.70275, -74.01180 40.7..."
...,...,...
331,248,"LINESTRING (-73.99741 40.72164, -73.99389 40.7..."
332,249,"LINESTRING (-74.00203 40.71159, -74.00252 40.7..."
333,250,"LINESTRING (-74.00296 40.71334, -74.00150 40.7..."
334,251,"LINESTRING (-74.00799 40.70899, -74.00720 40.7..."


# -----calculate distance between road and point-----

In [8]:
#find the 1 closest road for each point
def find_close_road(point, roads_df):
    close_road = None
    min_distance = float('inf')
    
    for idx, road in roads_df.iterrows():
        distance = point.distance(road['geometry'])
        if distance < min_distance:
            min_distance = distance
            close_road = road['id']
    return close_road

In [9]:
#apply above method to all points in df_point
df_point['close_road'] = df_point.apply(lambda row: find_close_road(row['geometry'], df_road), axis=1)

In [10]:
def find_closest_road(point, roads_df, min_distance):
    closest_road_id = []

    for idx, road in roads_df.iterrows():
        distance = point.distance(road['geometry'])
        if distance < min_distance:
            closest_road_id.append(road['id'])

    return closest_road_id

In [11]:
df_point['closest_road_id'] = df_point.apply(lambda row: find_closest_road(row['geometry'], df_road, 0.0004), axis=1)

In [12]:
df_point

,id,geometry,close_road,closest_road_id
0,0,POINT (-74.00698 40.74748),119,"[119, 170]"
1,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]"
2,2,POINT (-74.00721 40.74697),120,"[120, 170]"
3,3,POINT (-74.00722 40.74722),119,"[119, 120, 170]"
4,4,POINT (-74.00728 40.74667),120,"[120, 121, 170]"
...,...,...,...,...
2295,2295,POINT (-73.97976 40.71559),234,"[33, 234]"
2296,2296,POINT (-73.97987 40.71361),30,"[30, 35, 235]"
2297,2297,POINT (-73.98387 40.72077),221,[221]
2298,2298,POINT (-73.97780 40.71833),199,"[199, 235]"


# -----block file-----

In [13]:
def blockInfo(shapefile):
    gdf = gpd.read_file(shapefile)
    block_info_list = []
    for idx, row in gdf.iterrows():
        block_id = row['id'] 
        polygon = row['geometry'] 
        corners = list(polygon.exterior.coords)
        block_info = {
            'blockID': block_id,
            'corners': corners
        }
        block_info_list.append(block_info)
    return block_info_list

In [14]:
def blockIdentify(hydrantCenter, blocks):
    hydrantPoint = Point(hydrantCenter)
    for block in blocks:
        blockCoords = block['corners'] 
        blockPolygon = Polygon(blockCoords)
        if hydrantPoint.within(blockPolygon):
            hydrantBlockID = block['blockID'] 
            return hydrantBlockID
    return None

In [15]:
blocks = blockInfo("block_location/blocksagain.shp")
df_point['block_id'] = df_point.apply(lambda row: blockIdentify(row['geometry'], blocks), axis=1)

In [16]:
# Check if the entire dataframe contains NA values
is_na_anywhere = df_point.isna().any().any()

print("Dataframe contains NA values:", is_na_anywhere)

Dataframe contains NA values: True


In [17]:
df_point

,id,geometry,close_road,closest_road_id,block_id
0,0,POINT (-74.00698 40.74748),119,"[119, 170]",1.0
1,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0
2,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0
3,3,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0
4,4,POINT (-74.00728 40.74667),120,"[120, 121, 170]",3.0
...,...,...,...,...,...
2295,2295,POINT (-73.97976 40.71559),234,"[33, 234]",505.0
2296,2296,POINT (-73.97987 40.71361),30,"[30, 35, 235]",901.0
2297,2297,POINT (-73.98387 40.72077),221,[221],437.0
2298,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0


# -----Hydrant Class-----

In [18]:
class Hydrant:
    
    def __init__(self, id, x, y, block_id, closest_road_id, close_road): # constructor
        
        # From shapefile
        self.pointID = id
        self.x = x
        self.y = y
        self.blockID = block_id
        self.roadID = closest_road_id
        self.closeRoadID = close_road
        
        # Will initialize once into network generation
        self.branchID = 0
        self.neighbor = [] # List to store all the neighbors of this hydrant
        self.edge = [] # Dict to store edge information
        
        self.elevation = 0

# -----Edge Class-----

In [67]:
class Edge:
    
    def __init__(self, id, p1, p2, length): # constructor
    
        self.edgeID = id
        self.p1 = p1 # id
        self.p2 = p2 # id
        self.length = length
        self.cross_road_id = 0
        self.angle_with_road = 0
        self.intersection_angle = 0
        self.cost = 0

# -----convert the above dataframe to hydrant object-----

In [20]:
# Create an empty list to store the 'hydrant' instances
hydrant_instances = []

# Iterate through each row in the DataFrame and create 'hydrant' instances
for index, row in df_point.iterrows():
    # Get the Point object from the 'geometry' column
    point = row['geometry']
    
    # Extract 'x' and 'y' coordinates from the Point object
    x, y = point.x, point.y
    
    # Create 'hydrant' instance and add it to the list
    hydrant_obj = Hydrant(row['id'], x, y, row['block_id'], row['closest_road_id'], row['close_road'])
    hydrant_instances.append(hydrant_obj)

In [21]:
hydrant_instances[2].blockID

2.0

In [22]:
# Function to calculate the Euclidean distance between two points
def distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# -----Triangulation-----

In [23]:
# Extract x and y coordinates from hydrant instances
hydrant_points = [(hydrant.x, hydrant.y) for hydrant in hydrant_instances]

# Convert the list of points to a numpy array
points = np.array(hydrant_points)

# Perform Delaunay triangulation
triangulation = Delaunay(points)

# -----Filter out edges with different roadID-----

# -----Filter out edges cross Road that exceed the maximum-----

# -----Filter cross block with different roadID-----

In [24]:
# set the maximum length
max_edge_length = 0.002

In [25]:
#two functions altered for the closest point
def edge_in_block(hydrant1, hydrant2):
    return hydrant1.blockID == hydrant2.blockID

def blocked_by_block(hydrant1, hydrant2):
    if edge_in_block(hydrant1, hydrant2):
        if hydrant1.closeRoadID == hydrant2.closeRoadID:
            return False  
        return True
    else:
        return False

In [26]:
# Set to store filtered edges (edges that don't exceed the maximum length)
filtered_edges_road_cross = set()
edge_set_pro = []

In [27]:
# track edgeID
edge_id = 0

In [28]:
# Loop through each triangle (simplex)
for simplex in triangulation.simplices:
    
    # Get the indices of the points forming the edges of the triangle
    edge1 = frozenset([simplex[0], simplex[1]])
    edge2 = frozenset([simplex[1], simplex[2]])
    edge3 = frozenset([simplex[0], simplex[2]])

    # Get the actual points from the 'points' list using the indices
    point1, point2, point3 = points[simplex[0]], points[simplex[1]], points[simplex[2]]
    
    # Get the actual points from the 'hydrant_instances' list using the indices
    hydrant1, hydrant2, hydrant3 = hydrant_instances[simplex[0]], hydrant_instances[simplex[1]], hydrant_instances[simplex[2]]
    
    # Calculate the edge lengths
    edge1_length = distance(point1, point2)
    edge2_length = distance(point2, point3)
    edge3_length = distance(point1, point3)
        
    # Check each closest road ID for each point in the triangle
    for road_id in df_point.at[simplex[0], 'closest_road_id']:
        if edge1_length <= max_edge_length \
            and road_id in df_point.at[simplex[1], 'closest_road_id'] \
            and not blocked_by_block(hydrant1, hydrant2):
            filtered_edges_road_cross.add(edge1)
            edge_obj = Edge(edge_id, simplex[0], simplex[1], edge1_length)
            edge_set_pro.append(edge_obj)
            edge_id += 1
            hydrant1.neighbor.append(hydrant2)
            hydrant1.edge.append(edge_obj)
            break  # Break the loop if an edge is found for this road_id
    for road_id in df_point.at[simplex[1], 'closest_road_id']:
        if edge2_length <= max_edge_length \
            and road_id in df_point.at[simplex[2], 'closest_road_id'] \
            and not blocked_by_block(hydrant2, hydrant3):
            filtered_edges_road_cross.add(edge2)
            edge_obj = Edge(edge_id, simplex[1], simplex[2], edge2_length)
            edge_set_pro.append(edge_obj)
            edge_id += 1
            hydrant2.neighbor.append(hydrant3)
            hydrant2.edge.append(edge_obj)
            break
    for road_id in df_point.at[simplex[0], 'closest_road_id']:
        if edge3_length <= max_edge_length \
            and road_id in df_point.at[simplex[2], 'closest_road_id'] \
            and not blocked_by_block(hydrant1, hydrant3):
            filtered_edges_road_cross.add(edge3)
            edge_obj = Edge(edge_id, simplex[0], simplex[2], edge3_length)
            edge_set_pro.append(edge_obj)
            edge_id += 1
            hydrant1.neighbor.append(hydrant3)
            hydrant3.edge.append(edge_obj)
            break

In [29]:
# -----Create a GeoDataFrame for filtered edges-----

# Create a list of LineString objects for filtered edges
lines = []
for edge in filtered_edges_road_cross:
    edge_points = list(edge)
    line = LineString(np.array(points)[edge_points])
    lines.append(line)

# Create a GeoDataFrame from the LineString objects
gdf_filtered_edges_road_cross = gpd.GeoDataFrame(geometry=lines)

# Save the GeoDataFrame as a shapefile
gdf_filtered_edges_road_cross.to_file('filtered_edges_road_block.shp')

# -----Calculate angle cross road for each edge-----

In [30]:
def calculate_angle(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return math.degrees(math.atan2(dy, dx))

In [31]:
def unit_vector(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    length = np.sqrt(dx**2 + dy**2)
    if length > 0:
        return dx / length, dy / length
    else:
        return 0, 0

In [32]:
def find_edge_crossing_road(edges, roads_df):
    for edge in edges:
        road_id = None
        min_distance = float('inf')
        intersection_angle = 0  # Initialize the angle at the intersection
        
        # Get the actual points from the 'points' list using the indices
        point1, point2 = points[edge.p1], points[edge.p2]

        # Calculate the unit direction vector of the edge
        edge_vector = unit_vector(point1, point2)

        # Calculate the angle between the edge and each road
        for _, road_row in roads_df.iterrows():
            road_geometry = road_row['geometry']

            # Assuming road_geometry is a LINESTRING object containing coordinates of the road
            road_coords = road_geometry.coords[:]
            road_angle = calculate_angle(road_coords[0], road_coords[-1])

            # Calculate the unit direction vector of the road
            road_vector = unit_vector(road_coords[0], road_coords[-1])

            # Calculate the angle between the edge and the road
            angle_diff = np.arccos(np.clip(np.dot(edge_vector, road_vector), -1.0, 1.0))

            # Convert the angle from radians to degrees
            angle_diff_degrees = np.degrees(angle_diff)

            # Check if this road is closer than previous ones
            if angle_diff_degrees < min_distance:
                min_distance = angle_diff_degrees
                road_id = road_row['id']
                intersection_angle = road_angle

        # Now you have the road_id that the edge crosses, the angle between the edge and the road,
        # and the angle at the intersection of the edge and the road.
        # You can store this information in the edge object if desired.
        edge.cross_road_id = road_id
        edge.angle_with_road = min_distance
        edge.intersection_angle = intersection_angle


In [33]:
find_edge_crossing_road(edge_set_pro, df_road)

In [34]:
# Define a function to create a list of dictionaries containing the relevant attributes of each Edge object
def edge_attributes_to_list(edges):
    edge_list = []
    for edge in edges:
        edge_dict = {
            'edgeID': edge.edgeID,
            'p1': edge.p1,
            'p2': edge.p2,
            'length': edge.length,
            'cross_road_id': edge.cross_road_id,
            'angle_with_road': edge.angle_with_road,
            'intersection_angle': edge.intersection_angle
        }
        edge_list.append(edge_dict)
    return edge_list

# Call the function to convert the Edge objects to a list of dictionaries
edge_list = edge_attributes_to_list(edge_set_pro)

# Create a DataFrame from the list of dictionaries
edge_df = pd.DataFrame(edge_list)

# Display the DataFrame containing edge information
edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle
0,0,293,312,0.001844,189,0.033167,-103.984768
1,1,312,304,0.000686,243,0.105671,75.330291
2,2,293,304,0.001158,221,0.016852,-103.480157
3,3,307,304,0.000651,45,0.977769,146.990835
4,4,312,304,0.000686,243,0.105671,75.330291
...,...,...,...,...,...,...,...
8777,8777,445,436,0.000762,180,2.150902,-123.486169
8778,8778,436,441,0.000576,248,0.822384,-20.160698
8779,8779,445,441,0.000855,3,0.077049,-80.134193
8780,8780,445,435,0.000629,110,5.426604,-130.537332


In [35]:
# Helper function to normalize angles to the range of 0 to 90 degrees
def normalize_angle_0_to_90(angle):
    while angle < 0:
        angle += 360
    while angle >= 90:
        angle -= 90
    return angle

# Normalize the intersection angles in the pandas Series
edge_df['intersection_angle'] = edge_df['intersection_angle'].apply(normalize_angle_0_to_90)

In [36]:
edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle
0,0,293,312,0.001844,189,0.033167,76.015232
1,1,312,304,0.000686,243,0.105671,75.330291
2,2,293,304,0.001158,221,0.016852,76.519843
3,3,307,304,0.000651,45,0.977769,56.990835
4,4,312,304,0.000686,243,0.105671,75.330291
...,...,...,...,...,...,...,...
8777,8777,445,436,0.000762,180,2.150902,56.513831
8778,8778,436,441,0.000576,248,0.822384,69.839302
8779,8779,445,441,0.000855,3,0.077049,9.865807
8780,8780,445,435,0.000629,110,5.426604,49.462668


In [37]:
is_zero_in_intersection = (edge_df['intersection_angle'] == 0).any()

In [38]:
is_zero_in_intersection

False

# -----Elevation-----

In [39]:
elevation_shapefile_path = 'hydrant_locations_with_rasterdata/hydrant_locations_with_rasterdata.shp'
first_hydrant_raster_gdf = gpd.read_file(elevation_shapefile_path)

In [40]:
second_elevation_shapefilepath = 'second_hydrants_with_raster/second_hydrants_with_raster.shp'
second_hydrant_raster_gdf = gpd.read_file(second_elevation_shapefilepath)

In [41]:
def create_hydrants_elev_dict(first_hydrant_raster_gdf, second_hydrant_raster_gdf): 

  hydrants_to_elev = {}

  for index, row in first_hydrant_raster_gdf.iterrows():
    hydrant_point = row['geometry']
    if pd.isna(row['SAMPLE_1']): #if the value is not in the first shapefile DEM, check second DEM file 
        corresponding_elev = second_hydrant_raster_gdf.loc[index, 'ELEV1'] #call the corresponding elevation from second DEM file 
        hydrants_to_elev[(hydrant_point.x, hydrant_point.y)] = corresponding_elev #update the dictionary value 
    else:
        hydrants_to_elev[(hydrant_point.x, hydrant_point.y)] = row['SAMPLE_1']
  return hydrants_to_elev

In [42]:
elevation_data = create_hydrants_elev_dict(first_hydrant_raster_gdf, second_hydrant_raster_gdf)

In [43]:
# Create a new column 'elevation' in the df_point DataFrame and set it to None initially
df_point['elevation'] = None

# Iterate through the df_point DataFrame and update the 'elevation' column with the corresponding elevation from the dictionary
for index, row in df_point.iterrows():
    geometry = row['geometry']
    elevation = elevation_data.get((geometry.x, geometry.y), None)
    df_point.at[index, 'elevation'] = elevation
    hydrant_instances[index].elevation = elevation

In [45]:
elevation_data

{(-74.00697762006345, 40.74747950102119): 5.269999980926514,
 (-74.0072172653229, 40.747217389018665): 6.840000152587891,
 (-74.00720977640854, 40.74697399930203): 8.09000015258789,
 (-74.00722100978008, 40.74722113347584): 6.760000228881836,
 (-74.00728466555212, 40.746670698270535): 7.440000057220459,
 (-74.00788003424358, 40.74658083129824): 7.800000190734863,
 (-74.00645714051556, 40.747243600218916): 7.519999980926514,
 (-74.00642344040097, 40.746629509241565): 6.920000076293945,
 (-74.00799236795895, 40.74603788500729): 6.309999942779541,
 (-74.0075205663544, 40.74587312889141): 5.210000038146973,
 (-74.00754677755465, 40.74558480568863): -0.759999990463257,
 (-74.00808597938843, 40.745494938716334): 6.980000019073486,
 (-74.0055097928493, 40.74712377758919): 8.300000190734863,
 (-74.00494812427245, 40.74726232250481): 7.480000019073486,
 (-74.00527014758984, 40.74674558741411): 3.240000009536743,
 (-74.00589172748154, 40.74670814284232): 7.090000152587891,
 (-74.0056146376503, 4

In [46]:
df_point

,id,geometry,close_road,closest_road_id,block_id,elevation
0,0,POINT (-74.00698 40.74748),119,"[119, 170]",1.0,5.27
1,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.84
2,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09
3,3,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.76
4,4,POINT (-74.00728 40.74667),120,"[120, 121, 170]",3.0,7.44
...,...,...,...,...,...,...
2295,2295,POINT (-73.97976 40.71559),234,"[33, 234]",505.0,11.9
2296,2296,POINT (-73.97987 40.71361),30,"[30, 35, 235]",901.0,21.5
2297,2297,POINT (-73.98387 40.72077),221,[221],437.0,23.959999
2298,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0,9.1


# -----cost function-----

In [47]:
# sorry I know this is repetitive but I cannot find any other way at this point
def find_max_min_slope(edges):
    max_slope = float('-inf')
    min_slope = float('inf')

    for edge in edges:
        point_i = points[edge.p1]
        point_j = points[edge.p2]
        hydrant_i = hydrant_instances[edge.p1]
        hydrant_j = hydrant_instances[edge.p2]
        rise = hydrant_j.elevation - hydrant_i.elevation
        run = distance(point_i, point_j)
        slope = rise / run

        # Check for maximum slope
        if slope > max_slope:
            max_slope = slope

        # Check for minimum slope
        if slope < min_slope:
            min_slope = slope

    return max_slope, min_slope

In [48]:
max_length = edge_df['length'].max()
min_length = edge_df['length'].min()
max_angle = edge_df['intersection_angle'].max()
min_angle = edge_df['intersection_angle'].min()
max_slope, min_slope = find_max_min_slope(edge_set_pro)

In [49]:
def normalize_min_max(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

In [58]:
def costAngle(edge):
    return normalize_min_max(edge.intersection_angle, min_angle, max_angle)

def costLength(edge):
    point_i = points[edge.p1]
    point_j = points[edge.p2]
    hydrant_i = hydrant_instances[edge.p1]
    hydrant_j = hydrant_instances[edge.p2]
    return normalize_min_max(distance(point_i, point_j), 0, max_edge_length)

def costSlope(edge):
    hydrant_j = df_point.iloc[edge.p2]
    hydrant_i = df_point.iloc[edge.p1]
    rise = hydrant_j.elevation - hydrant_i.elevation
    run = edge.length
    slope = rise/run
    if slope < 0:
        return 1
    return normalize_min_max(slope, 0, max_slope)

# -----Genetic Algorithm-----

In [59]:
# Define the cost function using the given formula
def cost_function(αL, αS, α𝛳, edge):
    return αL * costLength(edge) + αS * costSlope(edge) + α𝛳 * costAngle(edge)

In [60]:
# Define the fitness function
def fitness(αL, αS, α𝛳, edges):
    total_cost = sum(cost_function(αL, αS, α𝛳, edge) for edge in edges)
    return total_cost

In [63]:
# Genetic Algorithm function
import random
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def genetic_algorithm(edges, population_size=50, num_generations=100, mutation_rate=0.1):
    num_edges = len(edges)

    # Generate an initial population of random individuals (chromosomes)
    population = np.random.rand(population_size, 3)  # [αL, αS, α𝛳]

    with ThreadPoolExecutor() as executor:
        for generation in range(num_generations):
            # Evaluate the fitness of each individual in the population using parallelism
            fitness_scores = list(executor.map(lambda ind: fitness(ind[0], ind[1], ind[2], edges), population))

            # Perform selection using indices of sorted fitness scores
            selected_indices = np.argsort(fitness_scores)[:population_size // 2]
            selected_population = population[selected_indices]

            # Perform crossover and mutation using NumPy vectorization
            parents = selected_population[np.random.choice(len(selected_population), size=(population_size, 2))]
            children = (parents[:, 0] + parents[:, 1]) / 2

            # Apply mutation using NumPy vectorization
            mask = np.random.rand(*children.shape) < mutation_rate
            mutation = np.random.uniform(-0.1, 0.1, size=children.shape)
            children += mask * mutation

            # Replace old population with new population
            population = children

    # Find the best individual
    best_index = np.argmin(fitness_scores)
    best_individual = {'αL': population[best_index, 0], 'αS': population[best_index, 1], 'α𝛳': population[best_index, 2]}

    return best_individual

In [64]:
best_weights = genetic_algorithm(edge_set_pro)

In [65]:
best_weights

{'αL': -0.6657995160591507,
 'αS': -0.9022471187293939,
 'α𝛳': 0.47820467928857113}

In [68]:
# Assuming you have already found the best weights
best_αL = best_weights['αL']
best_αS = best_weights['αS']
best_α𝛳 = best_weights['α𝛳']

# Calculate the cost for each edge using the best weights
edge_costs = [cost_function(best_αL, best_αS, best_α𝛳, edge) for edge in edge_set_pro]

# Add the calculated costs to the edge list as a new attribute
for edge, cost in zip(edge_set_pro, edge_costs):
    edge.cost = cost

In [69]:
edge_df['cost'] = edge_costs

In [70]:
edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost
0,0,293,312,0.001844,189,0.033167,76.015232,-1.179884
1,1,312,304,0.000686,243,0.105671,75.330291,0.170667
2,2,293,304,0.001158,221,0.016852,76.519843,-0.952010
3,3,307,304,0.000651,45,0.977769,56.990835,0.546357
4,4,312,304,0.000686,243,0.105671,75.330291,0.170667
...,...,...,...,...,...,...,...,...
8777,8777,445,436,0.000762,180,2.150902,56.513831,-0.991144
8778,8778,436,441,0.000576,248,0.822384,69.839302,-1.206129
8779,8779,445,441,0.000855,3,0.077049,9.865807,-0.731000
8780,8780,445,435,0.000629,110,5.426604,49.462668,-0.934739


In [139]:
sorted_grouped_edge_df = edge_df.groupby('p1', group_keys=False).apply(lambda x: x.sort_values(by='cost'))

In [140]:
sorted_grouped_edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost
1229,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531
1231,1231,1,0,0.000355,223,0.470305,47.093465,-0.770531
1233,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871
155,155,2,5,0.000777,4,1.109857,29.285695,-1.975491
446,446,2,5,0.000777,4,1.109857,29.285695,-1.975491
...,...,...,...,...,...,...,...,...
485,485,2298,1994,0.000812,166,0.896157,89.191648,-0.772399
481,481,2298,2011,0.000752,1,0.161768,29.744881,-0.102444
2443,2443,2298,2011,0.000752,1,0.161768,29.744881,-0.102444
474,474,2298,2014,0.000485,195,0.825450,74.378595,0.698870


In [141]:
sorted_grouped_edge_df = sorted_grouped_edge_df.drop_duplicates()

In [142]:
sorted_grouped_edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost
1229,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531
1231,1231,1,0,0.000355,223,0.470305,47.093465,-0.770531
1233,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871
155,155,2,5,0.000777,4,1.109857,29.285695,-1.975491
446,446,2,5,0.000777,4,1.109857,29.285695,-1.975491
...,...,...,...,...,...,...,...,...
485,485,2298,1994,0.000812,166,0.896157,89.191648,-0.772399
481,481,2298,2011,0.000752,1,0.161768,29.744881,-0.102444
2443,2443,2298,2011,0.000752,1,0.161768,29.744881,-0.102444
474,474,2298,2014,0.000485,195,0.825450,74.378595,0.698870


In [143]:
sorted_grouped_edge_df = sorted_grouped_edge_df.drop_duplicates(subset=['p1', 'p2', 'length', 'cross_road_id', 'angle_with_road', 'intersection_angle'])

In [144]:
sorted_grouped_edge_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost
1229,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531
1233,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871
155,155,2,5,0.000777,4,1.109857,29.285695,-1.975491
157,157,2,4,0.000312,189,0.115082,76.015232,-1.569332
7811,7811,2,7,0.000858,124,0.000242,66.341761,-1.318843
...,...,...,...,...,...,...,...,...
1608,1608,2297,2234,0.000020,71,0.225016,42.498705,-0.199366
480,480,2298,2012,0.000383,34,1.042716,72.862305,-1.125366
475,475,2298,1994,0.000812,166,0.896157,89.191648,-0.772399
481,481,2298,2011,0.000752,1,0.161768,29.744881,-0.102444


In [145]:
count_above_zero = (sorted_grouped_edge_df['cost'] > 0).sum()

In [146]:
count_above_zero

1002

In [147]:
no_above_zero_df = sorted_grouped_edge_df.loc[sorted_grouped_edge_df['cost'] <= 0]

In [148]:
no_above_zero_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost
1229,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531
1233,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871
155,155,2,5,0.000777,4,1.109857,29.285695,-1.975491
157,157,2,4,0.000312,189,0.115082,76.015232,-1.569332
7811,7811,2,7,0.000858,124,0.000242,66.341761,-1.318843
...,...,...,...,...,...,...,...,...
1569,1569,2297,2205,0.000856,194,0.658127,64.900005,-0.921377
1608,1608,2297,2234,0.000020,71,0.225016,42.498705,-0.199366
480,480,2298,2012,0.000383,34,1.042716,72.862305,-1.125366
475,475,2298,1994,0.000812,166,0.896157,89.191648,-0.772399


In [178]:
# Merge the points DataFrame for p1
filtered_edges_no_above_zero_df = pd.merge(no_above_zero_df, df_point, left_on='p1', right_on='id', how='left')

In [179]:
filtered_edges_no_above_zero_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost,id,geometry,close_road,closest_road_id,block_id,elevation
0,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.84
1,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.84
2,155,2,5,0.000777,4,1.109857,29.285695,-1.975491,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09
3,157,2,4,0.000312,189,0.115082,76.015232,-1.569332,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09
4,7811,2,7,0.000858,124,0.000242,66.341761,-1.318843,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,1569,2297,2205,0.000856,194,0.658127,64.900005,-0.921377,2297,POINT (-73.98387 40.72077),221,[221],437.0,23.959999
4851,1608,2297,2234,0.000020,71,0.225016,42.498705,-0.199366,2297,POINT (-73.98387 40.72077),221,[221],437.0,23.959999
4852,480,2298,2012,0.000383,34,1.042716,72.862305,-1.125366,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0,9.1
4853,475,2298,1994,0.000812,166,0.896157,89.191648,-0.772399,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0,9.1


In [180]:
# Duplicate the column with a different name
filtered_edges_no_above_zero_df['geometry_p1'] = filtered_edges_no_above_zero_df['geometry']
filtered_edges_no_above_zero_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost,id,geometry,close_road,closest_road_id,block_id,elevation,geometry_p1
0,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.84,POINT (-74.00722 40.74722)
1,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871,1,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.84,POINT (-74.00722 40.74722)
2,155,2,5,0.000777,4,1.109857,29.285695,-1.975491,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697)
3,157,2,4,0.000312,189,0.115082,76.015232,-1.569332,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697)
4,7811,2,7,0.000858,124,0.000242,66.341761,-1.318843,2,POINT (-74.00721 40.74697),120,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,1569,2297,2205,0.000856,194,0.658127,64.900005,-0.921377,2297,POINT (-73.98387 40.72077),221,[221],437.0,23.959999,POINT (-73.98387 40.72077)
4851,1608,2297,2234,0.000020,71,0.225016,42.498705,-0.199366,2297,POINT (-73.98387 40.72077),221,[221],437.0,23.959999,POINT (-73.98387 40.72077)
4852,480,2298,2012,0.000383,34,1.042716,72.862305,-1.125366,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0,9.1,POINT (-73.97780 40.71833)
4853,475,2298,1994,0.000812,166,0.896157,89.191648,-0.772399,2298,POINT (-73.97780 40.71833),199,"[199, 235]",444.0,9.1,POINT (-73.97780 40.71833)


In [181]:
# Merge the points DataFrame for p2
filtered_edges_no_above_zero_df = pd.merge(filtered_edges_no_above_zero_df, df_point, left_on='p2', right_on='id', how='left')

In [182]:
filtered_edges_no_above_zero_df

,edgeID,p1,p2,length,cross_road_id,angle_with_road,intersection_angle,cost,id_x,geometry_x,...,closest_road_id_x,block_id_x,elevation_x,geometry_p1,id_y,geometry_y,close_road_y,closest_road_id_y,block_id_y,elevation_y
0,1229,1,0,0.000355,223,0.470305,47.093465,-0.770531,1,POINT (-74.00722 40.74722),...,"[119, 120, 170]",2.0,6.84,POINT (-74.00722 40.74722),0,POINT (-74.00698 40.74748),119,"[119, 170]",1.0,5.27
1,1233,1,3,0.000005,169,1.881181,46.881181,-0.170871,1,POINT (-74.00722 40.74722),...,"[119, 120, 170]",2.0,6.84,POINT (-74.00722 40.74722),3,POINT (-74.00722 40.74722),119,"[119, 120, 170]",2.0,6.76
2,155,2,5,0.000777,4,1.109857,29.285695,-1.975491,2,POINT (-74.00721 40.74697),...,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697),5,POINT (-74.00788 40.74658),170,"[121, 170]",NaN,7.8
3,157,2,4,0.000312,189,0.115082,76.015232,-1.569332,2,POINT (-74.00721 40.74697),...,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697),4,POINT (-74.00728 40.74667),120,"[120, 121, 170]",3.0,7.44
4,7811,2,7,0.000858,124,0.000242,66.341761,-1.318843,2,POINT (-74.00721 40.74697),...,"[120, 170]",2.0,8.09,POINT (-74.00721 40.74697),7,POINT (-74.00642 40.74663),120,[120],2.0,6.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,1569,2297,2205,0.000856,194,0.658127,64.900005,-0.921377,2297,POINT (-73.98387 40.72077),...,[221],437.0,23.959999,POINT (-73.98387 40.72077),2205,POINT (-73.98424 40.71999),199,"[199, 221]",457.0,25.18
4851,1608,2297,2234,0.000020,71,0.225016,42.498705,-0.199366,2297,POINT (-73.98387 40.72077),...,[221],437.0,23.959999,POINT (-73.98387 40.72077),2234,POINT (-73.98388 40.72078),221,[221],437.0,23.91
4852,480,2298,2012,0.000383,34,1.042716,72.862305,-1.125366,2298,POINT (-73.97780 40.71833),...,"[199, 235]",444.0,9.1,POINT (-73.97780 40.71833),2012,POINT (-73.97743 40.71823),199,"[199, 235]",445.0,8.87
4853,475,2298,1994,0.000812,166,0.896157,89.191648,-0.772399,2298,POINT (-73.97780 40.71833),...,"[199, 235]",444.0,9.1,POINT (-73.97780 40.71833),1994,POINT (-73.97780 40.71752),235,[235],465.0,9.97


In [183]:
# List of column names to keep
columns_to_keep = ['edgeID', 'id_x', 'id_y', 'geometry_x', 'geometry_y', 'cost']

# Keep only the specified columns
filtered_edges_no_above_zero_df = filtered_edges_no_above_zero_df.loc[:, columns_to_keep]

In [184]:
filtered_edges_no_above_zero_df

,edgeID,id_x,id_y,geometry_x,geometry_y,cost
0,1229,1,0,POINT (-74.00722 40.74722),POINT (-74.00698 40.74748),-0.770531
1,1233,1,3,POINT (-74.00722 40.74722),POINT (-74.00722 40.74722),-0.170871
2,155,2,5,POINT (-74.00721 40.74697),POINT (-74.00788 40.74658),-1.975491
3,157,2,4,POINT (-74.00721 40.74697),POINT (-74.00728 40.74667),-1.569332
4,7811,2,7,POINT (-74.00721 40.74697),POINT (-74.00642 40.74663),-1.318843
...,...,...,...,...,...,...
4850,1569,2297,2205,POINT (-73.98387 40.72077),POINT (-73.98424 40.71999),-0.921377
4851,1608,2297,2234,POINT (-73.98387 40.72077),POINT (-73.98388 40.72078),-0.199366
4852,480,2298,2012,POINT (-73.97780 40.71833),POINT (-73.97743 40.71823),-1.125366
4853,475,2298,1994,POINT (-73.97780 40.71833),POINT (-73.97780 40.71752),-0.772399


In [185]:
# Create LineString geometries for both p1 and p2
filtered_edges_no_above_zero_df['geometry'] = filtered_edges_no_above_zero_df.apply(lambda row: LineString([row['geometry_x'], row['geometry_y']]), axis=1)

In [186]:
filtered_edges_no_above_zero_df

,edgeID,id_x,id_y,geometry_x,geometry_y,cost,geometry
0,1229,1,0,POINT (-74.00722 40.74722),POINT (-74.00698 40.74748),-0.770531,LINESTRING (-74.0072172653229 40.7472173890186...
1,1233,1,3,POINT (-74.00722 40.74722),POINT (-74.00722 40.74722),-0.170871,LINESTRING (-74.0072172653229 40.7472173890186...
2,155,2,5,POINT (-74.00721 40.74697),POINT (-74.00788 40.74658),-1.975491,LINESTRING (-74.00720977640854 40.746973999302...
3,157,2,4,POINT (-74.00721 40.74697),POINT (-74.00728 40.74667),-1.569332,LINESTRING (-74.00720977640854 40.746973999302...
4,7811,2,7,POINT (-74.00721 40.74697),POINT (-74.00642 40.74663),-1.318843,LINESTRING (-74.00720977640854 40.746973999302...
...,...,...,...,...,...,...,...
4850,1569,2297,2205,POINT (-73.98387 40.72077),POINT (-73.98424 40.71999),-0.921377,LINESTRING (-73.98386708301707 40.720766081186...
4851,1608,2297,2234,POINT (-73.98387 40.72077),POINT (-73.98388 40.72078),-0.199366,LINESTRING (-73.98386708301707 40.720766081186...
4852,480,2298,2012,POINT (-73.97780 40.71833),POINT (-73.97743 40.71823),-1.125366,LINESTRING (-73.97780125392501 40.718332506285...
4853,475,2298,1994,POINT (-73.97780 40.71833),POINT (-73.97780 40.71752),-0.772399,LINESTRING (-73.97780125392501 40.718332506285...


In [209]:
gdf = filtered_edges_no_above_zero_df.loc[:, ['geometry']]

In [210]:
gdf

,geometry
0,LINESTRING (-74.0072172653229 40.7472173890186...
1,LINESTRING (-74.0072172653229 40.7472173890186...
2,LINESTRING (-74.00720977640854 40.746973999302...
3,LINESTRING (-74.00720977640854 40.746973999302...
4,LINESTRING (-74.00720977640854 40.746973999302...
...,...
4850,LINESTRING (-73.98386708301707 40.720766081186...
4851,LINESTRING (-73.98386708301707 40.720766081186...
4852,LINESTRING (-73.97780125392501 40.718332506285...
4853,LINESTRING (-73.97780125392501 40.718332506285...


In [214]:
geometry = gdf['geometry'].apply(LineString)

In [215]:
gdf = gpd.GeoDataFrame(gdf, geometry=geometry)

In [216]:
# Define the output shapefile path
output_shapefile = 'GA_no_above_zero_lines.shp'

# Save the GeoDataFrame to a shapefile
gdf.to_file(output_shapefile)